In [ ]:
import pandas as pd
import numpy as np

df=pd.read_csv("Hamburger.csv")

In [ ]:
df.isna().sum()

In [ ]:
from sklearn.utils.discovery import all_estimators
# 다음 4개에 대해서 진행! 
estimators=all_estimators(type_filter='regressor')
for i in estimators:
    if "Ridge" in i[0] :
        print(i)

In [ ]:
from sklearn.linear_model import Ridge          # Ridge 회귀분석
from sklearn.kernel_ridge import KernelRidge
from sklearn.linear_model import RidgeCV
from sklearn.linear_model import BayesianRidge

from sklearn.model_selection import GridSearchCV # 최적의 파라미터를 찾기 위한 그리드서치 및 기타 기능 import
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [ ]:
df.head(3)

In [ ]:
feature=df[["sodium","sugar","total_fat","protein"]]
target=df["calories"]

In [ ]:
# Ridge : https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.Ridge.html#sklearn.linear_model.Ridge
test_max=0
train_max=0
for i in range(1000):
    x_train,x_test,y_train,y_test=train_test_split(feature,target,test_size=0.3,random_state=i)
    model=Ridge(random_state=i)
    model.fit(x_train,y_train)
    res1 = model.score(x_train,y_train)
    res2 = model.score(x_test,y_test)
    if np.abs(res1-res2) < 0.01 and res1 > train_max:
        train_max=res1
        test_max=res2
        print(i,train_max, test_max)

In [ ]:
x_train,x_test,y_train,y_test=train_test_split(feature,target,test_size=0.3,random_state=28)
model=GridSearchCV(Ridge(random_state=28, max_iter=1000),param_grid={"alpha":np.arange(0.1,1,0.1),
                                                      "solver": [ 'auto', 'svd', 'cholesky', 'lsqr', 'sparse_cg', 'sag', 'saga'],
                                                      "fit_intercept":[True,False]},cv=5).fit(x_train, y_train)

In [18]:
res=pd.DataFrame(model.cv_results_)
res[res["rank_test_score"]==1][["params","mean_test_score"]]

def save_model(model,filename):
    import joblib
    import os
    model_dir='./model/'
    model_filename=model_dir+f'{filename}.pkl'
    if not os.path.exists(model_dir):
        os.makedirs(model_dir)
    joblib.dump(model.best_estimator_, model_filename)
save_model(model,"Ridge")

In [19]:
# KernelRidge : https://scikit-learn.org/stable/modules/generated/sklearn.kernel_ridge.KernelRidge.html#sklearn.kernel_ridge.KernelRidge
test_max=0
train_max=0
for i in range(1000):
    x_train,x_test,y_train,y_test=train_test_split(feature,target,test_size=0.3,random_state=i)
    model=KernelRidge()
    model.fit(x_train,y_train)
    res1 = model.score(x_train,y_train)
    res2 = model.score(x_test,y_test)
    if np.abs(res1-res2) < 0.01 and res1 > train_max:
        train_max=res1
        test_max=res2
        print(i,train_max, test_max)

2 0.8636351236770505 0.8586224905219906
28 0.8650270699714239 0.857151481852169
154 0.8650497201420827 0.8567467433536906
588 0.8650684035308158 0.8555943121110731
872 0.8651218775571519 0.8555103822919929


In [22]:
x_train, x_test, y_train, y_test = train_test_split(feature, target, test_size=0.3, random_state=872)
param_grid = {
    'alpha': np.arange(0.1, 1, 0.1),
    'kernel': ['linear', 'poly', 'rbf', 'sigmoid', 'cosine'],  # 선택한 커널에 따라 추가 파라미터도 설정 가능
    'gamma': [0.1, 0.01, 0.001],  # rbf, poly, sigmoid 커널에 사용
    'degree': [2, 3, 4],  # poly 커널에 사용
}
model = GridSearchCV(KernelRidge(), param_grid=param_grid, cv=5)
model.fit(x_train, y_train)
# 최적의 하이퍼 파라미터 출력
print("Best hyperparameters:", model.best_params_)
# 테스트 데이터로 모델 평가
score = model.score(x_test, y_test)
print("Model score on test data:", score)


c:\ProgramData\anaconda3\envs\EXAM_ML\lib\site-packages\sklearn\linear_model\_ridge.py:248: LinAlgWarning: Ill-conditioned matrix (rcond=4.60931e-17): result may not be accurate.
  dual_coef = linalg.solve(K, y, assume_a="pos", overwrite_a=False)
c:\ProgramData\anaconda3\envs\EXAM_ML\lib\site-packages\sklearn\linear_model\_ridge.py:248: LinAlgWarning: Ill-conditioned matrix (rcond=6.4966e-17): result may not be accurate.
  dual_coef = linalg.solve(K, y, assume_a="pos", overwrite_a=False)
c:\ProgramData\anaconda3\envs\EXAM_ML\lib\site-packages\sklearn\linear_model\_ridge.py:248: LinAlgWarning: Ill-conditioned matrix (rcond=5.81415e-17): result may not be accurate.
  dual_coef = linalg.solve(K, y, assume_a="pos", overwrite_a=False)
c:\ProgramData\anaconda3\envs\EXAM_ML\lib\site-packages\sklearn\linear_model\_ridge.py:248: LinAlgWarning: Ill-conditioned matrix (rcond=6.31273e-17): result may not be accurate.
  dual_coef = linalg.solve(K, y, assume_a="pos", overwrite_a=False)
c:\ProgramDat

Best hyperparameters: {'alpha': 0.5, 'degree': 2, 'gamma': 0.1, 'kernel': 'poly'}
Model score on test data: 0.9026375352059494


In [23]:
save_model(model,"KernelRidge")

In [24]:
# RidgeCV : https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.RidgeCV.html#sklearn.linear_model.RidgeCV
test_max=0
train_max=0
for i in range(1000):
    x_train,x_test,y_train,y_test=train_test_split(feature,target,test_size=0.3,random_state=i)
    model=RidgeCV()
    model.fit(x_train,y_train)
    res1 = model.score(x_train,y_train)
    res2 = model.score(x_test,y_test)
    if np.abs(res1-res2) < 0.01 and res1 > train_max:
        train_max=res1
        test_max=res2
        print(i,train_max, test_max)

2 0.8645626457860155 0.8590183212020497
28 0.8662447921819565 0.856878583220491


In [33]:
x_train, x_test, y_train, y_test = train_test_split(feature, target, test_size=0.3, random_state=28)
param_grid = {
    'alphas': [0.1,0.01,0.001,0.0001,0.00001], 
    'fit_intercept': [True,False], 
    'gcv_mode': ['auto','svd','eigen'], 
}
model = GridSearchCV(RidgeCV(), param_grid=param_grid, cv=5)
model.fit(x_train, y_train)
# 최적의 하이퍼 파라미터 출력
print("Best hyperparameters:", model.best_params_)
# 테스트 데이터로 모델 평가
score = model.score(x_test, y_test)
print("Model score on test data:", score)
save_model(model,"RidgeCV")

Best hyperparameters: {'alphas': 1e-05, 'fit_intercept': True, 'gcv_mode': 'auto'}
Model score on test data: 0.8569702616585497


In [34]:
# BayesianRidge : https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.BayesianRidge.html#sklearn.linear_model.BayesianRidge
test_max=0
train_max=0
for i in range(1000):
    x_train,x_test,y_train,y_test=train_test_split(feature,target,test_size=0.3,random_state=i)
    model=BayesianRidge()
    model.fit(x_train,y_train)
    res1 = model.score(x_train,y_train)
    res2 = model.score(x_test,y_test)
    if np.abs(res1-res2) < 0.01 and res1 > train_max:
        train_max=res1
        test_max=res2
        print(i,train_max, test_max)

2 0.8645580005990545 0.858945523750401
28 0.8662405319397876 0.8567541791093402


In [38]:
x_train, x_test, y_train, y_test = train_test_split(feature, target, test_size=0.3, random_state=28)
param_grid = {
    'tol': [1e-4, 1e-5, 1e-6],  # 수렴 기준
    'alpha_1': [1e-7, 1e-6, 1e-5],  # alpha_1 값의 후보
    'alpha_2': [1e-7, 1e-6, 1e-5],  # alpha_2 값의 후보
    'lambda_1': [1e-7, 1e-6, 1e-5],  # lambda_1 값의 후보
    'lambda_2': [1e-7, 1e-6, 1e-5],  # lambda_2 값의 후보
    'alpha_init': [None, 1e-8, 1e-7],  # alpha_init 값의 후보
    'lambda_init': [None, 1e-8, 1e-7],  # lambda_init 값의 후보
    'compute_score': [True, False],  # 로그 마진 우도를 계산할지 여부
    'fit_intercept': [True, False],  # 절편을 계산할지 여부
}

model = GridSearchCV(BayesianRidge(max_iter=10000, verbose=True), param_grid=param_grid, cv=4)
model.fit(x_train, y_train)
# 최적의 하이퍼 파라미터 출력
print("Best hyperparameters:", model.best_params_)
# 테스트 데이터로 모델 평가
score = model.score(x_test, y_test)
print("Model score on test data:", score)

Convergence after  4  iterations
Convergence after  4  iterations
Convergence after  4  iterations
Convergence after  4  iterations
Convergence after  5  iterations
Convergence after  5  iterations
Convergence after  5  iterations
Convergence after  5  iterations
Convergence after  5  iterations
Convergence after  5  iterations
Convergence after  5  iterations
Convergence after  5  iterations
Convergence after  3  iterations
Convergence after  3  iterations
Convergence after  3  iterations
Convergence after  3  iterations
Convergence after  3  iterations
Convergence after  4  iterations
Convergence after  4  iterations
Convergence after  4  iterations
Convergence after  4  iterations
Convergence after  4  iterations
Convergence after  4  iterations
Convergence after  4  iterations
Convergence after  3  iterations
Convergence after  3  iterations
Convergence after  3  iterations
Convergence after  3  iterations
Convergence after  3  iterations
Convergence after  4  iterations
Convergenc

In [39]:
save_model(model,"BayesianRidge")